# Parallelizing the `for` loop

Manual matrix inversion and sparsity have led to great performance gains, but more can be accomplished. Perhaps the greatest source of potential is the single `for` loop that runs through each line in the network. There is no need for one process to handle every line in series, but the way the code is currently written precludes parallelization.

I begin the re-write by deleting old versions of the temporal instanton code. One of the greatest things about version control is that you don't need to be sentimental.

The files `TemporalInstanton3` and `PowerFlow` are the only ones I currrently use. Everything else contains duplicate or inferior code.

Can you parallelize a `for` loop when each loop accesses a common variable?

In [58]:
addprocs()

4-element Array{Int64,1}:
 6
 7
 8
 9

In [2]:
workers()

4-element Array{Int64,1}:
 2
 3
 4
 5

In [60]:
@everywhere f(x) = x^2
x = [i for i in 1:5]
@time pmap(f,x); # parallel
@time [f(i) for i in x]; # single-core

   4.001 milliseconds (749 allocations: 59408 bytes)
  20.694 microseconds (16 allocations: 592 bytes)


Parallel version is faster (but takes more space). Great! So this is what I'm supposed to do. I can start with an array of empty vectors and fill each with an instanton solution.

In [64]:
# works!
solutions = SharedArray(Int64,4)
@time @sync @parallel for i = 1:4
    solutions[i] = i;
end
solutions

4-element SharedArray{Int64,1}:
 1
 2
 3
 4

27.588 milliseconds (4927 allocations: 362 KB)


Though I love the idea of using a SharedArray to parallelize, I shouldn't rely on an [experimental, Unix-only feature](http://julia.readthedocs.org/en/latest/manual/parallel-computing/#shared-arrays-experimental).

In [65]:
# in this case, single-core is far superior
@time solutions = [i for i in 1:4]

4-element Array{Int64,1}:
 1
 2
 3
 4

 1.322 microseconds (1 allocation: 96 bytes)


In [66]:
rmprocs(workers())

:ok